In [1]:
import seaborn as sb
import pandas as pd
import matplotlib.pyplot as plt
from pandas import Grouper
from pandas import DataFrame
import csv
import pickle
import math

/home/jonathan/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df = pd.read_csv("/home/jonathan/tesis/Data_Temperatura_Lluvia/2020_Accidentalidad.csv",sep=';')#,index_col=['fecha'], parse_dates=True)
DataAcc = df.drop(columns=['localizacion','numero','estado_meteorológico','tipo_vehiculo','tipo_persona','rango_edad','sexo','cod_lesividad','lesividad','coordenada_x_utm','coordenada_y_utm','positiva_alcohol','positiva_droga'])
DataAcc['ds'] = DataAcc['fecha'] + '/' + DataAcc['hora']
DataAcc['ds'] = pd.to_datetime(DataAcc['ds'], format='%d/%m/%Y/%H:%M:%S')
DataAcc = DataAcc.drop(columns=['fecha','hora'])
DataAcc = DataAcc.set_index('ds')
DataAcc = DataAcc.sort_index()
DataAcc

,num_expediente,cod_distrito,distrito,tipo_accidente
ds,,,,
2020-01-01 01:00:00,2020S000014,3.0,RETIRO,Colisión fronto-lateral
2020-01-01 01:00:00,2020S000004,15.0,CIUDAD LINEAL,Choque contra obstáculo fijo
2020-01-01 01:00:00,2020S000004,15.0,CIUDAD LINEAL,Choque contra obstáculo fijo
2020-01-01 01:00:00,2020S000004,15.0,CIUDAD LINEAL,Choque contra obstáculo fijo
2020-01-01 01:00:00,2020S000014,3.0,RETIRO,Colisión fronto-lateral
...,...,...,...,...
2020-12-31 21:55:00,2020S019513,19.0,VICÁLVARO,Atropello a persona
2020-12-31 22:00:00,2020S019516,12.0,USERA,Choque contra obstáculo fijo
2020-12-31 22:25:00,2020S019517,13.0,PUENTE DE VALLECAS,Choque contra obstáculo fijo


In [3]:
# df = pd.read_csv("/home/jonathan/tesis/Data_Temperatura_Lluvia/2020_Accidentalidad.csv",sep=';')#,index_col=['fecha'], parse_dates=True)
# DataAcc = df.drop(columns=['localizacion','numero','estado_meteorológico','tipo_vehiculo','tipo_persona','rango_edad','sexo','cod_lesividad','lesividad','coordenada_x_utm','coordenada_y_utm','positiva_alcohol','positiva_droga'])
# DataAcc['fecha'] = pd.to_datetime(DataAcc['fecha'], format='%d/%m/%Y')
# DataAcc = DataAcc.set_index('fecha')
# DataAcc = DataAcc.sort_index()
# DataAcc

In [4]:
DataAcc = DataAcc.loc[((DataAcc['cod_distrito'] == 16) | (DataAcc['cod_distrito'] == 8)) # | (DataAcc['cod_distrito'] == 5)
                       & ((DataAcc['tipo_accidente'] != 'Caída') & (DataAcc['tipo_accidente'] != 'Otro'))]
DataAcc['Accidente'] = True
DataAcc = DataAcc.drop_duplicates(subset = ['num_expediente'])

/tmp/ipykernel_11283/1593683072.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DataAcc['Accidente'] = True


In [5]:
# # Observar hora exacta de accidente
data1=DataAcc.loc['2020-08-24':'2020-08-24']
data1.head(60)

,num_expediente,cod_distrito,distrito,tipo_accidente,Accidente
ds,,,,,
2020-08-24 01:00:00,2020S011555,8.0,FUENCARRAL-EL PARDO,Choque contra obstáculo fijo,True
2020-08-24 01:45:00,2020S011551,16.0,HORTALEZA,Choque contra obstáculo fijo,True
2020-08-24 03:00:00,2020S011552,8.0,FUENCARRAL-EL PARDO,Choque contra obstáculo fijo,True


In [6]:
DataAcc = DataAcc.reset_index()

In [7]:
def ModTime(ds):
    date = pd.to_datetime(ds)
    hora = date.hour
    minuto = date.minute
    if (hora != 23) & (minuto >= 25):   # Revisar si esta bien esta condición
        date = date.replace(hour=hora+1,minute=0)
    else:
        date = date.replace(minute=0)
    return (date)
DataAcc['ds'] = DataAcc['ds'].apply(ModTime)
DataAcc = DataAcc.drop_duplicates(subset = ['ds']) # Contabilizar en una nueva columna las repeticiones del index

In [8]:
DataAcc = DataAcc.set_index('ds')

In [9]:
Prueba = DataAcc.loc['2020-08-22':'2020-08-26']
Prueba

,num_expediente,cod_distrito,distrito,tipo_accidente,Accidente
ds,,,,,
2020-08-23 02:00:00,2020S011546,16.0,HORTALEZA,Atropello a persona,True
2020-08-24 01:00:00,2020S011555,8.0,FUENCARRAL-EL PARDO,Choque contra obstáculo fijo,True
2020-08-24 02:00:00,2020S011551,16.0,HORTALEZA,Choque contra obstáculo fijo,True
2020-08-24 03:00:00,2020S011552,8.0,FUENCARRAL-EL PARDO,Choque contra obstáculo fijo,True
2020-08-25 07:00:00,2020S011642,16.0,HORTALEZA,Colisión fronto-lateral,True
2020-08-25 14:00:00,2020S011643,16.0,HORTALEZA,Colisión lateral,True
2020-08-26 12:00:00,2020S011686,8.0,FUENCARRAL-EL PARDO,Alcance,True
2020-08-26 20:00:00,2020S011757,8.0,FUENCARRAL-EL PARDO,Alcance,True
2020-08-26 22:00:00,2020S011694,8.0,FUENCARRAL-EL PARDO,Alcance,True


In [10]:
DataAcc = DataAcc.reset_index()

In [11]:
def ds1(ds):
    global fila    
    date = pd.to_datetime(ds)    
    mes = date.month
    dia = date.day
    hora = date.hour
    try:
        if (hora != 23):
            date2 = date.replace(hour=hora+1,minute=0)
        else:
            date2 = date.replace(day=dia+1,hour=0)  
    except:
            date2 = date.replace(month=mes+1, day=1, hour=0) 
    
#     date3 = date.replace(hour=hora+2,minute=0)
    
    
    consulta1 = DataAcc.loc[fila+1]    
    celdaDs1 = consulta1['ds']
    celdaAcc1 = consulta1['Accidente']
    
    
    if fila < (len(DataAcc)-2):
        fila += 1
#     consulta2 = DataAcc.loc[fila+2]
#     celdaDs2 = consulta2['ds']
#     celdaAcc2 = consulta2['Accidente']
    
    if date2 == celdaDs1:
        try:
            if (hora < 22):
                date = date.replace(hour=hora+2,minute=0)
            elif (hora == 22):
                date = date.replace(day=dia+1,hour=0)
            elif (hora == 23):
                date = date.replace(day=dia+1,hour=1)
        except:            
            if (hora == 22):
                date = date.replace(month=mes+1, day=1, hour=0) 
            elif (hora == 23):
                date = date.replace(month=mes+1, day=1, hour=1) 
            
#         if date3 == celdaDs2:
#             if celdaAcc2 == True:
#             date = date.replace(hour=hora+2,minute=0)
    else:                
        try:
            if (hora != 23):   # Revisar si esta bien esta condición
                date = date.replace(hour=hora+1,minute=0)
            else:
                date = date.replace(day=dia+1,hour=0)  
        except:
            date = date.replace(month=mes+1, day=1, hour=0)  
    
    return (date)

fila = 0
DataAcc['ds1'] = DataAcc['ds'].apply(ds1)

In [12]:
len(DataAcc)-2

1171

In [13]:
# def ds1(ds):
#     date = pd.to_datetime(ds)
#     mes = date.month
#     dia = date.day
#     hora = date.hour    
#     try:
#         if (hora != 23):   # Revisar si esta bien esta condición
#             date = date.replace(hour=hora+1,minute=0)
#         else:
#             date = date.replace(day=dia+1,hour=0)  
#     except:
#         date = date.replace(month=mes+1, day=1, hour=0)  
#     return (date)
# DataAcc['ds1'] = DataAcc['ds'].apply(ds1)

def ds2(ds):
    date = pd.to_datetime(ds)
    mes = date.month
    dia = date.day
    hora = date.hour    
    try:
        if (hora < 22):   # Revisar si esta bien esta condición
            date = date.replace(hour=hora+2,minute=0)
        elif (hora == 22):
            date = date.replace(day=dia+1,hour=0)
        elif (hora == 23):
            date = date.replace(day=dia+1,hour=1)
    except:
        if (hora == 22):
            date = date.replace(month=mes+1, day=1, hour=0) 
        elif (hora == 23):
            date = date.replace(month=mes+1, day=1, hour=1) 
    return (date)
DataAcc['ds2'] = DataAcc['ds'].apply(ds2)

In [14]:
DataAcc
# DataAcc = DataAcc.set_index('ds')
# data1=DataAcc.loc['2020-08-01':'2020-08-05']
# data1

,ds,num_expediente,cod_distrito,distrito,tipo_accidente,Accidente,ds1,ds2
0,2020-01-01 01:00:00,2020S000002,16.0,HORTALEZA,Choque contra obstáculo fijo,True,2020-01-01 02:00:00,2020-01-01 03:00:00
1,2020-01-01 10:00:00,2020S000142,8.0,FUENCARRAL-EL PARDO,Choque contra obstáculo fijo,True,2020-01-01 11:00:00,2020-01-01 12:00:00
2,2020-01-01 12:00:00,2020S000206,8.0,FUENCARRAL-EL PARDO,Choque contra obstáculo fijo,True,2020-01-01 13:00:00,2020-01-01 14:00:00
3,2020-01-01 14:00:00,2020S000424,16.0,HORTALEZA,Choque contra obstáculo fijo,True,2020-01-01 15:00:00,2020-01-01 16:00:00
4,2020-01-01 16:00:00,2020S000061,16.0,HORTALEZA,Colisión fronto-lateral,True,2020-01-01 18:00:00,2020-01-01 18:00:00
...,...,...,...,...,...,...,...,...
1168,2020-12-31 13:00:00,2020S019483,8.0,FUENCARRAL-EL PARDO,Alcance,True,2020-12-31 14:00:00,2020-12-31 15:00:00
1169,2020-12-31 17:00:00,2020S019601,16.0,HORTALEZA,Colisión fronto-lateral,True,2020-12-31 18:00:00,2020-12-31 19:00:00
1170,2020-12-31 19:00:00,2020S019598,16.0,HORTALEZA,Choque contra obstáculo fijo,True,2020-12-31 21:00:00,2020-12-31 21:00:00
1171,2020-12-31 20:00:00,2020S019502,8.0,FUENCARRAL-EL PARDO,Atropello a persona,True,2020-12-31 22:00:00,2020-12-31 22:00:00


In [15]:
DataAcc['Accidente1'] = 1
DataAcc['Accidente2'] = 1
DataAcc = DataAcc.set_index('ds')
Acc_ds = DataAcc.loc[:,['Accidente']]
DataAcc = DataAcc.set_index('ds1')
Acc_ds1 = DataAcc.loc[:,['Accidente1']]
DataAcc = DataAcc.set_index('ds2')
Acc_ds2 = DataAcc.loc[:,['Accidente2']]

In [16]:
Acc_ds1
Data=Acc_ds1.loc['2020-08-24':'2020-08-24']
Data

,Accidente1
ds1,
2020-08-24 03:00:00,1
2020-08-24 04:00:00,1
2020-08-24 04:00:00,1


In [18]:
# Acc_ds.to_csv("Acc_ds.csv")
# Acc_ds1.to_csv("Acc_ds1.csv")
# Acc_ds2.to_csv("Acc_ds2.csv")

In [19]:
#############Prueba###############
# DataAccPrue2 = DataAcc1.loc[DataAcc1['tipo_accidente'] == 'Caída'] 
# DataAccPrue2.shape
# DataAccPrue2 = DataAcc1.loc[(DataAcc1['cod_distrito'] != 16) & (DataAcc1['cod_distrito'] != 8)] 
# DataAccPrue2.shape